### Imports

In [1]:
import pandas as pd
import numpy as np
from pyhhmm.gaussian import GaussianHMM
from pandas_datareader.data import DataReader
import matplotlib.pyplot as plt

### Structure Data

In [3]:
# Data Extraction
start_date = '2017-1-1'
end_date = '2022-6-1'

symbol = 'BTC-USD'
data = DataReader(name=symbol, data_source='yahoo', start=start_date, end=end_date)
data = data[['Open','High','Low','Adj Close']]
data

TypeError: string indices must be integers

In [ ]:
# Add Returns and Range
df = data.copy()
df['Returns'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
df['Range'] = (df['High'] / df['Low']) - 1
df.dropna(inplace=True)
print(f'Len: {len(df)}')
df.head()

In [ ]:
# Add Moving Average
df['MA_12'] = df['Adj Close'].rolling(window=12).mean()
df['MA_21'] = df['Adj Close'].rolling(window=21).mean()

In [ ]:
# Structure Data
X_train = df[['Returns', 'Range']].iloc[:500]
X_test = df[['Returns', 'Range']].iloc[500:]
save_df = df.iloc[500:]

print(f'Train Length: {len(X_train)}')
print(f'Test Length: {len(X_test)}')
print(f'X_train from: {X_train.head(1).index.item()}')
print(f'X_train to: {X_train.tail(1).index.item()}')
print(f'X_test from: {X_test.head(1).index.item()}')
print(f'X_test to: {X_test.tail(1).index.item()}')

### Train HMM

In [ ]:
model = GaussianHMM(n_states=4, covariance_type='full', n_emissions=2)
model.train([np.array(X_train.values)])
model.predict([X_train.values])[0][:10]

In [ ]:
# Make Prediction on Test Data
df_main = save_df.copy()
df_main.drop(columns=['High', 'Low'] ,inplace=True)

hmm_results = model.predict([X_test.values])[0]
df_main['HMM'] = hmm_results
df_main.head()

### Run Backtest

In [ ]:
# Add MA Signals
df_main.loc[df_main['MA_12'] > df_main['MA_21'], 'MA_Signal'] = 1
df_main.loc[df_main['MA_12'] <= df_main['MA_21'], 'MA_Signal'] = 0
df_main

In [ ]:
# Add HMM Signals
favourable_states = [1, 3]
hmm_values = df_main['HMM'].values
hmm_values = [1 if x in favourable_states else 0 for x in hmm_values]
df_main['HMM_Signal'] = hmm_values
df_main.iloc[:5]

In [ ]:
# Add Combined Signal
df_main['Main_Signal'] = 0
df_main.loc[(df_main['MA_Signal'] == 1) & (df_main['HMM_Signal'] == 1), 'Main_Signal'] = 1
df_main['Main_Signal'] = df_main['Main_Signal'].shift(1)

In [ ]:
# Benchmark Returns
df_main['lrets_bench'] = np.log(df_main['Adj Close'] / df_main['Adj Close'].shift(1))
df_main['bench_prod'] = df_main['lrets_bench'].cumsum()
df_main['bench_prod_exp'] = np.exp(df_main['bench_prod']) - 1

In [ ]:
# Strategy Returns
df_main['lrets_strat'] = np.log(df_main['Open'].shift(-1) / df_main['Open']) * df_main['Main_Signal']
df_main['lrets_prod'] = df_main['lrets_strat'].cumsum()
df_main['strat_prod_exp'] = np.exp(df_main['lrets_prod']) - 1

In [ ]:
# Review Results Table
df_main.dropna(inplace=True)
df_main.tail(5)

### Calculate Metrics

In [ ]:
# Sharpe Ratio
def sharpe_ratio(return_series):
    N = 365
    NSQRT = np.sqrt(N)
    rf = 0.01
    mean = return_series.mean() * N
    sigma = return_series.std() * NSQRT
    sharpe_ratio = round((mean - rf) / sigma, 2)
    return sharpe_ratio

In [ ]:
# Metrics
bench_rets = round(df_main['bench_prod_exp'].values[-1] * 100, 1)
strat_rets = round(df_main['strat_prod_exp'].values[-1] * 100, 1)

bench_sharpe = sharpe_ratio(df_main['lrets_bench'].values)
strat_sharpe = sharpe_ratio(df_main['lrets_strat'].values)

In [ ]:
# Print Metrics
print(f'Returns Benchmark: {bench_rets}%')
print(f'Returns Strategy: {strat_rets}%')
print('---- ---- ---- ---- ---- ---- ----')
print(f'Sharpe Benchmark: {bench_sharpe}')
print(f'Sharpe Strategy: {strat_sharpe}')

### Plot Results

In [ ]:
# Plot Equity Curves
fig = plt.figure(figsize=(18, 10))
plt.plot(df_main['bench_prod_exp'])
plt.plot(df_main['strat_prod_exp'])
plt.show()

### Save Data

In [ ]:
# Save Data
# df_main.to_csv('data/HMM-SPY.csv')

In [ ]:
print(X_test.values[-1])